In [ ]:
import math
import heapq
import pandas as pd

# Define the Cell class
class Cell:
    def __init__(self):
        self.parentnode1 = 0
        self.parentnode2= 0
        self.f = float('inf')
        self.g = float('inf')
        self.h = 0

# Grid size
griddf = pd.read_csv('grid.csv', header=None)
cellcolumn= griddf.shape[1]
cellrow = griddf.shape[0]

# Define processes that check if cells are in grid, unblocked and is the destination
def ingrid(row, column):
    return (row >= 0) and (row < cellrow) and (column >= 0) and (column < cellcolumn)

def unblocked(grid, row, column):
    return grid[row][column] == 1

def destination(row, column, dest):
    return row == dest[0] and column == dest[1]

# Calculate the heuristic value
def calculate_h_value(row, column, dest):
    return ((row - dest[0]) ** 2 + (column - dest[1]) ** 2) ** 0.5

# Site path from source to destination
def bestpath(cell_details, dest):
    print("The best path from the source to desination is as follows")
    path = []
    row = dest[0]
    column = dest[1]


    # Trace path using parent cells to define new rows
    while not (cell_details[row][column].parentnode1 == row and cell_details[row][column].parentnode2 == column):
        path.append((row, column))
        newrow = cell_details[row][column].parentnode1
        newcol = cell_details[row][column].parentnode2
        row = newrow
        column = newcol

    # Add the source cell to the path
    path.append((row, column))
    path.reverse()

    # Print the path
    for i in path:
        print("->", i, end=" ")
    print()

# Implement
def a_star_search(grid, src, dest):


    if not ingrid(src[0], src[1]) or not ingrid(dest[0], dest[1]):
        print("Source or destination is invalid")
        return

    if not unblocked(grid, src[0], src[1]) or not unblocked(grid, dest[0], dest[1]):
        print("Source or the destination is blocked")
        return


    # Initialize the closed list (visited cells)
    closedcells = [[False for _ in range(cellcolumn)] for _ in range(cellrow)]
    # Initialize the details of each cell
    cell_details = [[Cell() for _ in range(cellcolumn)] for _ in range(cellrow)]

    # Initialize the source cell
    a = src[0]
    b = src[1]
    cell_details[a][b].f = 0
    cell_details[a][b].g = 0
    cell_details[a][b].h = 0
    cell_details[a][b].parentnode1 = a
    cell_details[a][b].parentnode2 = b


    # List open cells list from source and flag for a gound destination
    opencells = []
    heapq.heappush(opencells, (0.0, a, b))
    destinationfound = False

    # Main loop of A* search algorithm
    while len(opencells) > 0:

        usednode = heapq.heappop(opencells)

        # Mark the cell as used
        a = usednode[1]
        b = usednode[2]
        closedcells[a][b] = True

        # For each direction, check possibilities
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for dir in directions:
            tempcell1 = a + dir[0]
            tempcell2 = b + dir[1]

            # If the new cell is in the grid, unblocked, and not visited and is the destination
            if ingrid(tempcell1, tempcell2) and unblocked(grid, tempcell1, tempcell2) and not closedcells[tempcell1][tempcell2]:
                if destination(tempcell1, tempcell2, dest):

                    # Set the parent of the destination cell and print path
                    cell_details[tempcell1][tempcell2].parentnode1 = a
                    cell_details[tempcell1][tempcell2].parentnode2 = b
                    print("Path to destination is available!")
                    bestpath(cell_details, dest)
                    found_dest = True


                    return
                else:
                    # Calculate the new f, g, and h values
                    newg = cell_details[a][b].g + 1.0
                    newh = calculate_h_value(tempcell1, tempcell2, dest)
                    newf = newg + newh

                    # If the cell is not in the open list or the new f value is smaller update its details
                    if cell_details[tempcell1][tempcell2].f == float('inf') or cell_details[tempcell1][tempcell2].f > newf:
                        heapq.heappush(opencells, (newf, tempcell1, tempcell2))
                        cell_details[tempcell1][tempcell2].f = newf
                        cell_details[tempcell1][tempcell2].g = newg
                        cell_details[tempcell1][tempcell2].h = newh
                        cell_details[tempcell1][tempcell2].parentnode1 = a
                        cell_details[tempcell1][tempcell2].parentnode2 = b


    if not destinationfound:
        print("Path unavailabe, due to blockages.")

def main():
    # Define the grid (1 for unblocked, 0 for blocked)
    grid = griddf.values.tolist()


#Pick Source and Destination Cell
    src = [1, 0]
    dest = [612,612]

    a_star_search(grid, src, dest)

#Bounding Condition
if __name__ == "__main__":
    main()
